# Tutorial 14: Multi-Agent Collaboration

In this tutorial, you'll build a team of specialized agents that collaborate on complex tasks using the **supervisor pattern**.

**What you'll learn:**
- How to create a supervisor agent that coordinates multiple specialists
- Defining specialized agent roles (researcher, coder, reviewer)
- State management for multi-agent systems with output accumulation
- Routing decisions between agents using structured output
- Synthesizing outputs from multiple agents into a final result

By the end, you'll have a working multi-agent system where a supervisor coordinates researcher, coder, and reviewer agents to complete complex tasks.

## Prerequisites

- Completed Tutorials 01-07 (Core Patterns)
- Understanding of StateGraph, nodes, and conditional edges
- Ollama running with a capable model (llama3.1:8b or larger recommended for tool calling)

## Why Multi-Agent Collaboration?

Single agents work well for simple tasks, but complex tasks often benefit from:

1. **Specialization**: Different agents excel at different tasks (research vs coding vs review)
2. **Quality Control**: Review agents can catch errors before final output
3. **Modularity**: Each agent can be improved independently
4. **Scalability**: Add new specialist agents as needed

The **supervisor pattern** uses a central coordinator that:
- Understands the overall task
- Decides which specialist should work next
- Tracks progress and knows when to stop

```
                     ┌─────────────────┐
                     │   Supervisor    │
                     │     Agent       │
                     └────────┬────────┘
                              │
          ┌───────────────────┼───────────────────┐
          │                   │                   │
          ▼                   ▼                   ▼
   ┌─────────────┐     ┌─────────────┐     ┌─────────────┐
   │  Researcher │     │   Coder     │     │  Reviewer   │
   └─────────────┘     └─────────────┘     └─────────────┘
          │                   │                   │
          └───────────────────┴───────────────────┘
                              │
                              ▼
                     ┌─────────────────┐
                     │   Synthesize    │
                     └─────────────────┘
```

## Step 1: Setup and Verify Environment

In [ ]:
# Verify our setup
from langgraph_ollama_local import LocalAgentConfig

config = LocalAgentConfig()
print(f"Ollama URL: {config.ollama.base_url}")
print(f"Model: {config.ollama.model}")
print("Setup verified!")

## Step 2: Define the Multi-Agent State

Our state needs to track:
- The task being worked on
- Which agent should run next (supervisor's decision)
- Accumulated outputs from all agents
- Iteration count to prevent infinite loops
- The final synthesized result

In [ ]:
from typing import Annotated, Literal
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
import operator


class MultiAgentState(TypedDict):
    """State schema for multi-agent collaboration."""
    
    # Conversation history - accumulates messages
    messages: Annotated[list, add_messages]
    
    # The task to complete
    task: str
    
    # Supervisor's routing decision
    next_agent: str
    
    # Accumulated agent outputs - uses operator.add to append
    agent_outputs: Annotated[list[dict], operator.add]
    
    # Iteration tracking
    iteration: int
    max_iterations: int
    
    # Final result
    final_result: str


print("State defined!")
print("\nKey fields:")
print("- agent_outputs: Uses operator.add to accumulate outputs from each agent")
print("- next_agent: Set by supervisor to route to the right specialist")

## Step 3: Create the Supervisor with Structured Output

The supervisor uses **structured output** to make routing decisions. This ensures reliable parsing of which agent should work next.

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage


class SupervisorDecision(BaseModel):
    """Structured output for supervisor routing."""
    
    next_agent: Literal["researcher", "coder", "reviewer", "FINISH"] = Field(
        description="The next agent to work, or FINISH if complete"
    )
    reasoning: str = Field(
        description="Brief explanation for this decision"
    )


SUPERVISOR_PROMPT = """You are a supervisor managing a team of specialized agents.

Your team:
- **researcher**: Analyzes requirements, gathers information, researches solutions
- **coder**: Writes code, implements solutions
- **reviewer**: Reviews work quality, checks for issues

Based on the task and progress, decide which agent should work next.
When the task is complete, respond with FINISH.

Guidelines:
- Start with researcher for tasks needing analysis
- Use coder when requirements are clear
- Use reviewer after code is written
- Be efficient - most tasks need 2-4 agent turns"""


def create_supervisor_node(llm):
    """Create a supervisor that routes work to agents."""
    
    # Use structured output for reliable parsing
    structured_llm = llm.with_structured_output(SupervisorDecision)
    
    def supervisor(state: MultiAgentState) -> dict:
        # Summarize progress
        outputs = state.get("agent_outputs", [])
        if outputs:
            progress = "\n".join([
                f"**{o['agent']}**: {o['output'][:300]}..."
                for o in outputs
            ])
        else:
            progress = "No work completed yet."
        
        messages = [
            SystemMessage(content=SUPERVISOR_PROMPT),
            HumanMessage(content=f"""Task: {state['task']}

Progress:
{progress}

Iteration {state['iteration'] + 1}/{state['max_iterations']}

Which agent should work next?""")
        ]
        
        decision = structured_llm.invoke(messages)
        
        return {
            "next_agent": decision.next_agent,
            "iteration": state["iteration"] + 1,
            "messages": [AIMessage(
                content=f"[Supervisor] Next: {decision.next_agent}. {decision.reasoning}"
            )]
        }
    
    return supervisor


print("Supervisor node creator defined!")

## Step 4: Create Specialized Agent Nodes

Each specialist agent has a focused role and system prompt. They all follow the same pattern but with different expertise.

In [ ]:
AGENT_PROMPTS = {
    "researcher": """You are a research agent. Your job is to:
- Analyze task requirements
- Research best practices and solutions
- Identify potential challenges

Provide clear, actionable analysis.""",

    "coder": """You are a coding agent. Your job is to:
- Write clean, documented code
- Implement solutions based on requirements
- Follow best practices

Write production-quality code with examples.""",

    "reviewer": """You are a review agent. Your job is to:
- Review code for correctness and style
- Check if requirements are met
- Provide constructive feedback

Be specific about strengths and improvements needed."""
}


def create_agent_node(llm, agent_name: str):
    """Create a specialized agent node."""
    
    system_prompt = AGENT_PROMPTS.get(agent_name, "You are a helpful assistant.")
    
    def agent(state: MultiAgentState) -> dict:
        # Build context from previous work
        outputs = state.get("agent_outputs", [])
        if outputs:
            context = "\n\n".join([
                f"**{o['agent']}**: {o['output']}"
                for o in outputs
            ])
        else:
            context = "No previous work."
        
        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=f"""Task: {state['task']}

Previous work:
{context}

Provide your contribution.""")
        ]
        
        response = llm.invoke(messages)
        
        return {
            "agent_outputs": [{
                "agent": agent_name,
                "output": response.content
            }],
            "messages": [AIMessage(
                content=f"[{agent_name.title()}] {response.content}"
            )]
        }
    
    return agent


print("Agent node creator defined!")
print("\nAvailable agent types:", list(AGENT_PROMPTS.keys()))

## Step 5: Create the Synthesize Node

When the supervisor decides the task is complete, this node combines all agent outputs into a final result.

In [ ]:
def synthesize_node(state: MultiAgentState) -> dict:
    """Combine all agent outputs into final result."""
    
    outputs = state.get("agent_outputs", [])
    
    if not outputs:
        return {"final_result": "No work was completed."}
    
    # Group by agent
    by_agent = {}
    for output in outputs:
        agent = output["agent"]
        if agent not in by_agent:
            by_agent[agent] = []
        by_agent[agent].append(output["output"])
    
    # Build formatted result
    parts = []
    for agent_name in ["researcher", "coder", "reviewer"]:
        if agent_name in by_agent:
            parts.append(f"## {agent_name.title()} Output")
            for output in by_agent[agent_name]:
                parts.append(output)
            parts.append("")
    
    return {
        "final_result": "\n\n".join(parts).strip(),
        "messages": [AIMessage(content="[System] Task completed.")]
    }


print("Synthesize node defined!")

## Step 6: Define the Routing Function

This function reads the supervisor's decision and routes to the appropriate agent or to synthesis.

In [ ]:
def route_supervisor(state: MultiAgentState) -> str:
    """Route based on supervisor's decision."""
    
    next_agent = state.get("next_agent", "")
    iteration = state.get("iteration", 0)
    max_iterations = state.get("max_iterations", 10)
    
    # Force completion at max iterations
    if iteration >= max_iterations:
        print(f"  [Router] Max iterations reached, going to synthesize")
        return "synthesize"
    
    # Route based on decision
    if next_agent == "FINISH":
        print(f"  [Router] Task complete, going to synthesize")
        return "synthesize"
    
    if next_agent.lower() in ["researcher", "coder", "reviewer"]:
        print(f"  [Router] Routing to {next_agent.lower()}")
        return next_agent.lower()
    
    # Default to synthesize
    print(f"  [Router] Unknown decision '{next_agent}', going to synthesize")
    return "synthesize"


print("Routing function defined!")

## Step 7: Build the Complete Graph

Now we assemble all the pieces into a working multi-agent graph.

In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain_ollama import ChatOllama

# Create LLM
llm = ChatOllama(
    model=config.ollama.model,
    base_url=config.ollama.base_url,
    temperature=0,  # Deterministic for routing
)

# Build the graph
workflow = StateGraph(MultiAgentState)

# Add nodes
workflow.add_node("supervisor", create_supervisor_node(llm))
workflow.add_node("researcher", create_agent_node(llm, "researcher"))
workflow.add_node("coder", create_agent_node(llm, "coder"))
workflow.add_node("reviewer", create_agent_node(llm, "reviewer"))
workflow.add_node("synthesize", synthesize_node)

# Add edges
workflow.add_edge(START, "supervisor")

# Supervisor routes to agents or synthesize
workflow.add_conditional_edges(
    "supervisor",
    route_supervisor,
    {
        "researcher": "researcher",
        "coder": "coder",
        "reviewer": "reviewer",
        "synthesize": "synthesize",
    }
)

# All agents return to supervisor
workflow.add_edge("researcher", "supervisor")
workflow.add_edge("coder", "supervisor")
workflow.add_edge("reviewer", "supervisor")

# Synthesize ends the graph
workflow.add_edge("synthesize", END)

# Compile
graph = workflow.compile()

print("Multi-agent graph compiled!")
print("\nGraph structure:")
print("  START -> supervisor")
print("  supervisor -> [researcher | coder | reviewer | synthesize]")
print("  researcher -> supervisor")
print("  coder -> supervisor")
print("  reviewer -> supervisor")
print("  synthesize -> END")

## Step 8: Run the Multi-Agent System

Let's test our multi-agent system with a real task!

In [ ]:
def run_task(task: str, max_iterations: int = 6):
    """Run a task through the multi-agent system."""
    
    print("="*60)
    print(f"Task: {task}")
    print("="*60)
    
    initial_state = {
        "messages": [HumanMessage(content=f"Task: {task}")],
        "task": task,
        "next_agent": "",
        "agent_outputs": [],
        "iteration": 0,
        "max_iterations": max_iterations,
        "final_result": "",
    }
    
    result = graph.invoke(initial_state)
    
    print("\n" + "="*60)
    print("FINAL RESULT")
    print("="*60)
    print(result["final_result"])
    print("\n" + "="*60)
    print(f"Completed in {result['iteration']} iterations")
    print(f"Agent outputs: {len(result['agent_outputs'])}")
    
    return result


# Test with a coding task
result = run_task(
    "Create a Python function to check if a string is a valid palindrome (ignoring spaces and case)"
)

## Step 9: Try Different Tasks

The multi-agent system can handle various types of tasks. Let's try a few more.

In [ ]:
# Research-heavy task
result2 = run_task(
    "Explain the best practices for error handling in Python async code",
    max_iterations=4
)

In [ ]:
# Implementation task
result3 = run_task(
    "Create a simple LRU cache class in Python with get and put methods",
    max_iterations=5
)

## Step 10: Using the Built-in Module

The `langgraph_ollama_local.agents` module provides a ready-to-use implementation:

In [ ]:
from langgraph_ollama_local.agents import (
    create_multi_agent_graph,
    run_multi_agent_task,
)

# Create the graph using the module
module_graph = create_multi_agent_graph(llm)

# Run a task
result = run_multi_agent_task(
    module_graph,
    "Write a Python decorator that logs function execution time",
    max_iterations=5
)

print("Final Result:")
print(result["final_result"])

## Complete Code

Here's the complete implementation in one cell for reference:

In [ ]:
# === Complete Multi-Agent Collaboration Implementation ===

from typing import Annotated, Literal
from typing_extensions import TypedDict
import operator

from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field

from langgraph_ollama_local import LocalAgentConfig


# === State ===
class MultiAgentState(TypedDict):
    messages: Annotated[list, add_messages]
    task: str
    next_agent: str
    agent_outputs: Annotated[list[dict], operator.add]
    iteration: int
    max_iterations: int
    final_result: str


# === Structured Output ===
class SupervisorDecision(BaseModel):
    next_agent: Literal["researcher", "coder", "reviewer", "FINISH"] = Field(
        description="Next agent or FINISH"
    )
    reasoning: str = Field(description="Brief explanation")


# === Prompts ===
SUPERVISOR_PROMPT = """You manage researcher, coder, and reviewer agents.
Decide who works next based on task and progress. Say FINISH when done."""

AGENT_PROMPTS = {
    "researcher": "Analyze requirements and research solutions.",
    "coder": "Write clean, documented code.",
    "reviewer": "Review quality and provide feedback.",
}


# === Node Creators ===
def create_supervisor(llm):
    structured_llm = llm.with_structured_output(SupervisorDecision)
    
    def supervisor(state):
        outputs = state.get("agent_outputs", [])
        progress = "\n".join([f"{o['agent']}: {o['output'][:200]}" for o in outputs]) or "None"
        
        decision = structured_llm.invoke([
            SystemMessage(content=SUPERVISOR_PROMPT),
            HumanMessage(content=f"Task: {state['task']}\nProgress: {progress}")
        ])
        
        return {"next_agent": decision.next_agent, "iteration": state["iteration"] + 1}
    
    return supervisor


def create_agent(llm, name):
    def agent(state):
        response = llm.invoke([
            SystemMessage(content=AGENT_PROMPTS[name]),
            HumanMessage(content=f"Task: {state['task']}")
        ])
        return {"agent_outputs": [{"agent": name, "output": response.content}]}
    return agent


def synthesize(state):
    outputs = state.get("agent_outputs", [])
    result = "\n\n".join([f"## {o['agent'].title()}\n{o['output']}" for o in outputs])
    return {"final_result": result}


# === Routing ===
def route(state):
    if state["iteration"] >= state["max_iterations"] or state["next_agent"] == "FINISH":
        return "synthesize"
    return state["next_agent"].lower() if state["next_agent"].lower() in AGENT_PROMPTS else "synthesize"


# === Build Graph ===
def build_multi_agent_graph():
    config = LocalAgentConfig()
    llm = ChatOllama(model=config.ollama.model, base_url=config.ollama.base_url, temperature=0)
    
    g = StateGraph(MultiAgentState)
    g.add_node("supervisor", create_supervisor(llm))
    g.add_node("researcher", create_agent(llm, "researcher"))
    g.add_node("coder", create_agent(llm, "coder"))
    g.add_node("reviewer", create_agent(llm, "reviewer"))
    g.add_node("synthesize", synthesize)
    
    g.add_edge(START, "supervisor")
    g.add_conditional_edges("supervisor", route, 
        {"researcher": "researcher", "coder": "coder", "reviewer": "reviewer", "synthesize": "synthesize"})
    g.add_edge("researcher", "supervisor")
    g.add_edge("coder", "supervisor")
    g.add_edge("reviewer", "supervisor")
    g.add_edge("synthesize", END)
    
    return g.compile()


# === Use ===
if __name__ == "__main__":
    graph = build_multi_agent_graph()
    result = graph.invoke({
        "messages": [], "task": "Write a function to reverse a linked list",
        "next_agent": "", "agent_outputs": [],
        "iteration": 0, "max_iterations": 5, "final_result": ""
    })
    print(result["final_result"])

## Key Concepts

| Concept | Description |
|---------|-------------|
| **Supervisor Pattern** | Central agent coordinates multiple specialists |
| **Structured Output** | Pydantic models ensure reliable routing decisions |
| **State Accumulation** | `operator.add` reducer collects all agent outputs |
| **Conditional Routing** | Supervisor decision determines next agent |
| **Agent Loop** | Agents return to supervisor until task is complete |
| **Max Iterations** | Safety limit prevents infinite loops |
| **Synthesis Node** | Combines all outputs into final result |

## What's Next

- [Tutorial 15: Hierarchical Agent Teams](15_hierarchical_teams.ipynb) - Nested teams with team supervisors
- [Tutorial 16: Subgraph Patterns](16_subgraphs.ipynb) - Composable, reusable graph components